
# CNN text classifcation model 
<hr>

In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import nltk
import random
from nltk.corpus import stopwords, twitter_samples
# from nltk.tokenize import TweetTokenizer
from sklearn.model_selection import KFold
from nltk.stem import PorterStemmer
from string import punctuation
from sklearn.preprocessing import OneHotEncoder
from tensorflow.keras.preprocessing.text import Tokenizer
import time

%config IPCompleter.greedy=True
%config IPCompleter.use_jedi=False
# nltk.download('twitter_samples')

In [8]:
corpus = pd.read_csv("places_names_labelled.csv", index_col = 0)
# corpus.label = corpus.label.astype(int)
print(corpus.shape)
corpus

(25161, 8)


,name,clean_name,place_id,vicinity,no_of_ratings,avg_rating,type_1,purpose
0,Colombo Swimming Club,colombo swimming club,ChIJETFLQ0FZ4joRwioIp4MnzHA,148 “Storm Lodge” Galle Road 3,1181.0,4.4,tourist_attraction,recreational
1,Storm lodge,storm lodge,ChIJGSU_sGtZ4joRiyZjgglUlPE,"148 “Storm Lodge” Galle Road 3, Colombo 00300",NaN,NaN,lodging,personal
5,Ayura,ayura,ChIJQwy6bEFZ4joRU9m1ylxsbMs,"142, Yathama Building, Galle Road, Colombo",16.0,4.5,jewelry_store,shopping
6,Baby Gallery Kurulapina,baby gallery kurulapina,ChIJR1dqFUFZ4joRsqACvaqRNws,"WR8X+655, Colombo",NaN,NaN,clothing_store,shopping
7,"Galle Face , Taj Hotel",galle face taj hotel,ChIJo015E0FZ4joRdGUr8gWxo4E,"138 Colombo - Galle Main Road, Colombo",2.0,5.0,restaurant,dining
...,...,...,...,...,...,...,...,...
44395,Ichiro Motoring - Wijerama,ichiro motoring wijerama,ChIJk3QsSX1a4joRDO-HD30GZT8,"High Level Road, Nugegoda",60.0,3.5,car_repair,personal
44396,Softlogic Glomark Supermarket,softlogic glomark supermarket,ChIJD436Ynxb4joR_tn_grvgpNs,"Nugegoda Delkanda Apartment, High Level Road, ...",1278.0,4.2,grocery_or_supermarket,shopping
44397,Softlogic Glomark Supermarket,softlogic glomark supermarket,ChIJD436Ynxb4joR_tn_grvgpNs,"Nugegoda Delkanda Apartment, High Level Road, ...",1278.0,4.2,grocery_or_supermarket,shopping
44398,Mirai Auto Land (Pvt) Ltd,mirai auto land pvt ltd,ChIJ6V8_E2da4joREMxUiWN0uVA,"Delkanda, 383 High Level Rd, Avissawella Road,...",52.0,4.4,car_repair,personal


In [9]:
corpus['purpose'].replace(corpus['purpose'].unique(), list(range(0, len(corpus['purpose'].unique()))), inplace=True)

In [10]:
corpus = corpus[~(corpus['purpose']==6)]

In [11]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(corpus,corpus['purpose'],test_size=0.2, random_state=32)


<hr>

A __standard model__ for document classification is to use (quoted from __Jason Brownlee__, the author of [machinelearningmastery.com](https://machinelearningmastery.com)):
>- Word Embedding: A distributed representation of words where different words that have a similar meaning (based on their usage) also have a similar representation.
>- Convolutional Model: A feature extraction model that learns to extract salient features from documents represented using a word embedding.
>- Fully Connected Model: The interpretation of extracted features in terms of a predictive output.


Therefore, the model is comprised of the following elements:
- __Input layer__ that defines the length of input sequences.
- __Embedding layer__ set to the size of the vocabulary and 100-dimensional real-valued representations.
- __Conv1D layer__ with 32 filters and a kernel size set to the number of words to read at once.
- __MaxPooling1D layer__ to consolidate the output from the convolutional layer.
- __Flatten layer__ to reduce the three-dimensional output to two dimensional for concatenation.

The CNN model is inspired by __Yoon Kim__ paper in his study on the use of Word Embedding + CNN for text classification. The hyperparameters we use based on his study are as follows:
- Transfer function: rectified linear.
- Kernel sizes: 1-8.
- Number of filters: 100.
- Dropout rate: 0.5.
- L2 Constraint: 3.
- Batch Size: 50.
- Update Rule: Adam

We will perform the best parameter using __grid search__ and 10-fold cross validation.

<hr>

In [12]:
# Define a function to compute the max length of sequence
def max_length(sequences):
    '''
    input:
        sequences: a 2D list of integer sequences
    output:
        max_length: the max length of the sequences
    '''
    max_length = 0
    for i, seq in enumerate(sequences):
        length = len(seq)
        if max_length < length:
            max_length = length
    return max_length

In [27]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

trunc_type='post'
padding_type='post'
oov_tok = "<UNK>"

# Separate the sentences and the labels
train_x = list(X_train.clean_name)
train_y = np.array(X_train.purpose)
test_x = list(X_test.clean_name)
test_y = np.array(X_test.purpose)

# Cleaning and Tokenization
tokenizer = Tokenizer(oov_token=oov_tok)
tokenizer.fit_on_texts(train_x)

print("Example of sentence: ", train_x[133])

# Turn the text into sequence
training_sequences = tokenizer.texts_to_sequences(train_x)
max_len = max_length(training_sequences)

print('Into a sequence of int:', training_sequences[133])

# Pad the sequence to have the same size
training_padded = pad_sequences(training_sequences, maxlen=max_len, padding=padding_type, truncating=trunc_type)
print('Into a padded sequence:', training_padded[133])

Example of sentence:  gateway asia travels
Into a sequence of int: [600, 131, 30]
Into a padded sequence: [600 131  30   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0]


In [28]:
# See the first 10 words in the vocabulary
word_index = tokenizer.word_index
for i, word in enumerate(word_index):
    print(word, word_index.get(word))
    if i==9:
        break
        
vocab_size = len(word_index)+1
print(vocab_size)

<UNK> 1
ltd 2
pvt 3
lanka 4
the 5
sri 6
colombo 7
of 8
and 9
bank 10
12898


In [34]:
from tensorflow.keras import regularizers
from tensorflow.keras.constraints import MaxNorm

def define_model(filters = 100, kernel_size = 3, activation='relu', input_dim = None, output_dim=300, max_length = None ):
    
    model = tf.keras.models.Sequential([
        
        tf.keras.layers.Embedding(input_dim=vocab_size, 
                                  output_dim=output_dim, 
                                  input_length=max_length, 
                                  input_shape=(max_length, )),
        
        tf.keras.layers.Conv1D(filters=filters, kernel_size = kernel_size, activation = activation, 
                               # set 'axis' value to the first and second axis of conv1D weights (rows, cols)
                               kernel_constraint= MaxNorm(max_value=3, axis=[0,1])),
        
        tf.keras.layers.MaxPool1D(2),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(10, activation=activation, 
                              # set axis to 0 to constrain each weight vector of length (input_dim,) in dense layer
                              kernel_constraint = MaxNorm( max_value=3, axis=0)),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(units=8, activation='softmax')
    ])
    
    model.compile(loss = 'sparse_categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
#     model.summary()
    return model

In [35]:
model_0 = define_model(input_dim=1000, max_length=100)
model_0.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_5 (Embedding)      (None, 100, 300)          3869400   
_________________________________________________________________
conv1d_4 (Conv1D)            (None, 98, 100)           90100     
_________________________________________________________________
max_pooling1d_4 (MaxPooling1 (None, 49, 100)           0         
_________________________________________________________________
flatten_4 (Flatten)          (None, 4900)              0         
_________________________________________________________________
dropout_8 (Dropout)          (None, 4900)              0         
_________________________________________________________________
dense_8 (Dense)              (None, 10)                49010     
_________________________________________________________________
dropout_9 (Dropout)          (None, 10)               

In [ ]:
class myCallback(tf.keras.callbacks.Callback):
    
    # Overide the method on_epoch_end() for our benefit
    def on_epoch_end(self, epoch, logs={}):
        if (logs.get('accuracy') > 0.93):
            print("\nReached 93% accuracy so cancelling training!")
            self.model.stop_training=True


callbacks = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', min_delta=0, 
                                             patience=20, verbose=2, 
                                             mode='auto', restore_best_weights=True)
# callbacks = myCallback()

In [37]:
# Parameter Initialization
trunc_type='post'
padding_type='post'
oov_tok = "<UNK>"
activations = ['relu', 'tanh']
filters = 1000
kernel_sizes = [1, 2, 3, 4]

columns = ['Activation', 'Filters', 'Acc']
record = pd.DataFrame(columns = columns)

# Separate the sentences and the labels
train_x = list(X_train.clean_name)
train_y = np.array(X_train.purpose)
test_x = list(X_test.clean_name)
test_y = np.array(X_test.purpose)

for activation in activations:

    for kernel_size in kernel_sizes:

        # encode data using
        # Cleaning and Tokenization
        tokenizer = Tokenizer(oov_token=oov_tok)
        tokenizer.fit_on_texts(train_x)

        # Turn the text into sequence
        training_sequences = tokenizer.texts_to_sequences(train_x)
        test_sequences = tokenizer.texts_to_sequences(test_x)

        max_len = max_length(training_sequences)

        # Pad the sequence to have the same size
        Xtrain = pad_sequences(training_sequences, maxlen=max_len, padding=padding_type, truncating=trunc_type)
        Xtest = pad_sequences(test_sequences, maxlen=max_len, padding=padding_type, truncating=trunc_type)

        word_index = tokenizer.word_index
        vocab_size = len(word_index)+1

        # Define the input shape
        model = define_model(filters, kernel_size, activation, input_dim=vocab_size, max_length=max_len)

        # Train the model
        model.fit(Xtrain, train_y, batch_size=50, epochs=60, verbose=2, 
                  callbacks=[callbacks], validation_data=(Xtest, test_y))

        # evaluate the model
        loss, acc = model.evaluate(Xtest, test_y, verbose=0)
        print('Test Accuracy: {}'.format(acc*100))
        
        parameters = [activation, kernel_size]
        entries = parameters + [acc]
        
        temp = pd.DataFrame([entries], columns=columns)
        record = record.append(temp, ignore_index=True)
        
        print()
        print(record)
        print()

Epoch 1/60
385/385 - 22s - loss: 1.5778 - accuracy: 0.3972 - val_loss: 1.1691 - val_accuracy: 0.5952
Epoch 2/60
385/385 - 24s - loss: 1.2312 - accuracy: 0.5264 - val_loss: 1.0677 - val_accuracy: 0.6118
Epoch 3/60
385/385 - 23s - loss: 1.0543 - accuracy: 0.5953 - val_loss: 1.0137 - val_accuracy: 0.6047
Epoch 4/60
385/385 - 24s - loss: 0.9245 - accuracy: 0.6303 - val_loss: 0.9935 - val_accuracy: 0.6538
Epoch 5/60
385/385 - 23s - loss: 0.8553 - accuracy: 0.6447 - val_loss: 0.9907 - val_accuracy: 0.6484
Epoch 6/60
385/385 - 23s - loss: 0.7728 - accuracy: 0.6950 - val_loss: 1.0723 - val_accuracy: 0.6910
Epoch 7/60
385/385 - 24s - loss: 0.7394 - accuracy: 0.7163 - val_loss: 1.0583 - val_accuracy: 0.6850
Epoch 8/60
385/385 - 24s - loss: 0.7019 - accuracy: 0.7296 - val_loss: 1.0550 - val_accuracy: 0.6850
Epoch 9/60
385/385 - 24s - loss: 0.6959 - accuracy: 0.7253 - val_loss: 1.1276 - val_accuracy: 0.6802
Epoch 10/60
385/385 - 26s - loss: 0.6731 - accuracy: 0.7314 - val_loss: 1.1334 - val_accura